### Load data

Use https://data.imf.org/?sk=9D6028D4-F14A-464C-A2F2-59B2CD424B85&sId=1390030341854 to download the data and load in the data. Here 2021M1 and export information is used

In [ ]:
import pandas as pd

# We use Jan 2021 data
trade_latest = pd.read_excel("export_2021m1.xlsx", skiprows=1, index_col=0, nrows=7)
trade_latest

In [ ]:
trade_latest.index

### Create network

In [ ]:
# We first convert the matrix data into pair data
trade_latest = pd.melt(trade_latest.reset_index(), id_vars='index')
trade_latest.columns = ['export_country', 'import_country', 'value']
trade_latest.head()

In [ ]:
trade_latest.dropna(inplace=True)
trade_latest.head()

In [ ]:
# Now we can use the pairs to create the graph:
#   (i) first two columns representing the nodes; 
#   (ii) last column representing the weight
# We also set the edge color based on 'export_country'

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

G = nx.DiGraph()  #create a directed graph instead

for index, row in trade_latest.iterrows():

    if row['export_country']=='United Kingdom':
        e_color='red'
    elif row['export_country']=='United States':
        e_color='blue'
    elif row['export_country']=='Canada':
        e_color='orange'
    else:
        e_color='grey'
    
    G.add_edge(row['export_country'], row['import_country'], weight=row['value'], color=e_color)

# remove isolated vertices (if any)    
remove = [node for node,degree in G.degree() if degree ==0]
G.remove_nodes_from(remove)

# settings
options = {
     'node_color': 'grey',
     'node_size': 1000,
     'alpha': 1,
     'connectionstyle': 'arc3, rad=0.1'}  #set the connection style

colors = nx.get_edge_attributes(G,'color').values()

weights = nx.get_edge_attributes(G,'weight').values()
weights = np.divide(list(weights),5000)  #adjust weights so that line widths are appropriate

# Reference on ConnectionStyle:
# https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.ConnectionStyle.html

### Draw graph

In [ ]:
# Spring layout

plt.subplots(figsize=(5,5))

pos=nx.spring_layout(G)

nx.draw(G, pos=pos, width=weights, edge_color=colors, **options)
nx.draw_networkx_labels(G,pos=pos, font_size=9)

# Alternatively, this will also work
#nx.draw_networkx(G, pos=pos, font_size=9, with_labels=True, width=weights, edge_color=colors, **options)

plt.tight_layout()
plt.axis('off');
plt.show()

# Reference:
# https://networkx.org/documentation/stable/reference/drawing.html

In [ ]:
# Random layout

plt.subplots(figsize=(5,5))
pos=nx.random_layout(G)
nx.draw(G, pos=pos, width=weights, edge_color=colors, **options)
nx.draw_networkx_labels(G, pos=pos, font_size=9)
plt.tight_layout()
plt.axis('off');
plt.show()

In [ ]:
# Circular layout

plt.subplots(figsize=(5,5))
pos=nx.circular_layout(G)
nx.draw(G, pos=pos, width=weights, edge_color=colors, **options)
nx.draw_networkx_labels(G, pos=pos, font_size=9)
plt.tight_layout()
plt.axis('off');
plt.show()

In [ ]:
# Circular layout - with additional code to change node size based on different measures

# Amend options to make allow customized node_size
options = {
     'node_color': 'grey',
     'alpha': 1,
     'connectionstyle': 'arc3, rad=0.1'}

plt.subplots(figsize=(5,5))

pos=nx.circular_layout(G)

d = dict(G.degree(weight="weight"))
#d = dict(nx.eigenvector_centrality(G))  #if using eigenvector_centrality
#d = dict(nx.betweenness_centrality(G))  #if using betweenness_centrality

nx.draw(G, pos=pos, 
        nodelist=list(d.keys()), 
        node_size=[v*0.05 for v in d.values()],  #multiply by factor to adjust size
        width=weights, edge_color=colors, **options)
nx.draw_networkx_labels(G, pos=pos, font_size=9)

# Alternatively, this will also work
#nx.draw_networkx(G, pos=pos, 
#                 nodelist=list(d.keys()), node_size=[v*0.05 for v in d.values()], 
#                 width=weights, edge_color=colors, 
#                 font_size=9, with_labels=True, **options)

plt.tight_layout()
plt.axis('off')
plt.show()

# References:
#   https://networkx.org/documentation/stable/reference/drawing.html
#   https://towardsdatascience.com/customizing-networkx-graphs-f80b4e69bedf

In [ ]:
# Save network graph for next part of assignment
import pickle

f = open("graph_pickle_latest", "wb")  # open file for writing in binary mode
pickle.dump(G, file = f)
f.close()